# Ejercicio 3 - Precios de Combustible

In [ ]:
import pandas as pd
import numpy as np
import requests

## Lectura de Datos

In [ ]:
#primero asigno el json convertido a dataframe en una variable
#después al explorar el dataset, veo que los valores de la columna ListaEESSPrecio son un diccionario de valores,


!pip install requests
import requests
import json

df_uno=pd.read_json("https://sedeaplicaciones.minetur.gob.es/ServiciosRESTCarburantes/PreciosCarburantes/EstacionesTerrestres/")
df_uno['ListaEESSPrecio']
df_uno.head(10)

,Fecha,ListaEESSPrecio,Nota,ResultadoConsulta
0,10/01/2022 14:46:55,"{'C.P.': '02250', 'Dirección': 'AVENIDA CASTIL...",Archivo de todos los productos en todas las es...,OK
1,10/01/2022 14:46:55,"{'C.P.': '02152', 'Dirección': 'CR CM-332, 46,...",Archivo de todos los productos en todas las es...,OK
2,10/01/2022 14:46:55,"{'C.P.': '02001', 'Dirección': 'CALLE PRINCIPE...",Archivo de todos los productos en todas las es...,OK
3,10/01/2022 14:46:55,"{'C.P.': '02001', 'Dirección': 'CALLE FEDERICO...",Archivo de todos los productos en todas las es...,OK
4,10/01/2022 14:46:55,"{'C.P.': '02001', 'Dirección': 'AVENIDA 1º DE ...",Archivo de todos los productos en todas las es...,OK
5,10/01/2022 14:46:55,"{'C.P.': '02001', 'Dirección': 'CALLE FEDERICO...",Archivo de todos los productos en todas las es...,OK
6,10/01/2022 14:46:55,"{'C.P.': '02005', 'Dirección': 'PASEO CUBA (LA...",Archivo de todos los productos en todas las es...,OK
7,10/01/2022 14:46:55,"{'C.P.': '02005', 'Dirección': 'AVENIDA MENÉND...",Archivo de todos los productos en todas las es...,OK
8,10/01/2022 14:46:55,"{'C.P.': '02005', 'Dirección': 'AVENIDA ESCRIT...",Archivo de todos los productos en todas las es...,OK
9,10/01/2022 14:46:55,"{'C.P.': '02005', 'Dirección': 'CL PASEO DE LA...",Archivo de todos los productos en todas las es...,OK


## Exploración y limpieza de datos



**Primera tabla**

In [ ]:
#que quiero dividir en varias columnas, utilizo el método json_normalize
df_uno=pd.json_normalize(df_uno['ListaEESSPrecio'])
df_uno.tail(10)

,C.P.,Dirección,Horario,Latitud,Localidad,Longitud (WGS84),Margen,Municipio,Precio Biodiesel,Precio Bioetanol,Precio Gas Natural Comprimido,Precio Gas Natural Licuado,Precio Gases licuados del petróleo,Precio Gasoleo A,Precio Gasoleo B,Precio Gasoleo Premium,Precio Gasolina 95 E10,Precio Gasolina 95 E5,Precio Gasolina 95 E5 Premium,Precio Gasolina 98 E10,Precio Gasolina 98 E5,Precio Hidrogeno,Provincia,Remisión,Rótulo,Tipo Venta,% BioEtanol,% Éster metílico,IDEESS,IDMunicipio,IDProvincia,IDCCAA
11398,50190,"RONDA CANAL IMPERIAL DE ARAGON, 8",L-V: 08:00-22:30; S: 10:00-22:30,"41,651444",GARRAPINILLOS,"-0,981111",N,Zaragoza,,,,,,"1,195",,"1,259",,"1,309",,,"1,519",,ZARAGOZA,dm,FAMILY ENERGY,P,"0,0","0,0",11519,8108,50,02
11399,50190,"AUTOVIA A-2 KM. 306,0",L-D: 07:00-23:00,"41,618389",GARRAPINILLOS,"-1,042444",I,Zaragoza,,,,,,"1,399",,"1,469",,"1,519",,,,,ZARAGOZA,OM,GALP,P,"0,0","0,0",1458,8108,50,02
11400,50190,"AUTOVIA A-2 KM. 305,65",L-D: 24H,"41,618083",GARRAPINILLOS,"-1,046583",I,Zaragoza,,,,,,"1,399",,"1,429",,"1,519",,,,,ZARAGOZA,OM,GALP,P,"0,0","0,0",1987,8108,50,02
11401,50692,"AUTOVIA A-68 KM. 258,25",L-V: 05:00-22:00; S-D: 08:00-22:00,"41,740389",VILLARRAPA,"-1,078944",D,Zaragoza,,,,,,"1,399","0,959","1,489",,"1,539",,,"1,659",,ZARAGOZA,OM,REPSOL,P,"0,0","0,0",8275,8108,50,02
11402,50197,"AVENIDA DIAGONAL PLAZA, 22",L-V: 06:00-22:00; S: 08:00-15:00,"41,636528",ZARAGOZA,"-0,990111",D,Zaragoza,,,,,,"1,419",,"1,489",,"1,539",,,"1,679",,ZARAGOZA,dm,REPSOL,P,"0,0","0,0",12289,8108,50,02
11403,50197,"RONDA FERIA DE MUESTRAS, DE LA, 20",L-D: 24H,"41,637944",ZARAGOZA,"-0,986389",D,Zaragoza,,,,,,"1,409","1,036",,,,,,,,ZARAGOZA,OM,IDS,P,"0,0","0,0",12472,8108,50,02
11404,50197,"POLIGONO CALLE TURIASO, 31",L-D: 24H,"41,645722",ZARAGOZA,"-0,974194",D,Zaragoza,,,,,,"1,359",,,,,,,,,ZARAGOZA,dm,NOIL,P,"0,0","0,0",13917,8108,50,02
11405,50800,"CARRETERA N-330A KM. 24,000",L-D: 06:00-22:00,"41,857000",ZUERA,"-0,794000",D,Zuera,,,,,,"1,399","1,001","1,489",,"1,519",,,"1,649",,ZARAGOZA,OM,REPSOL,P,"0,0","0,0",1452,8109,50,02
11406,50800,"CALLE AUSTRIA, 11,17",L-D: 24H,"41,883583",ZUERA,"-0,782833",D,Zuera,,,,,,"1,306","0,775","1,346",,"1,459",,,,,ZARAGOZA,dm,COOPERATIVA SAN LICER,P,"0,0","0,0",9085,8109,50,02
11407,50810,"CALLE COOPERATIVA, 2",L-D: 24H,"41,938750",ONTINAR DE SALZ,"-0,757611",D,Zuera,,,,,,"1,309","0,780","1,349",,"1,469",,,,,ZARAGOZA,dm,COOPERATIVA SAN ISIDRO,P,"0,0","0,0",9068,8109,50,02


In [ ]:
#exploramos todos los posibles valores de la columna IDCCAA porque es la que necesitaremos para unir las dos tablas
df_uno.IDCCAA.value_counts().reset_index()

,index,IDCCAA
0,01,2204
1,09,1482
2,10,1266
3,07,964
4,08,915
5,13,793
6,12,685
7,02,511
8,05,450
9,14,441


In [ ]:
df_uno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11408 entries, 0 to 11407
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   C.P.                                11408 non-null  object
 1   Dirección                           11408 non-null  object
 2   Horario                             11408 non-null  object
 3   Latitud                             11408 non-null  object
 4   Localidad                           11408 non-null  object
 5   Longitud (WGS84)                    11408 non-null  object
 6   Margen                              11408 non-null  object
 7   Municipio                           11408 non-null  object
 8   Precio Biodiesel                    11408 non-null  object
 9   Precio Bioetanol                    11408 non-null  object
 10  Precio Gas Natural Comprimido       11408 non-null  object
 11  Precio Gas Natural Licuado          11408 non-null  ob

**Segunda tabla**

In [ ]:
#quito las columnas que no me interesan
df_uno=df_uno.drop(columns=["Dirección", "Horario", "Remisión", "% BioEtanol", "% Éster metílico", "Rótulo", "Tipo Venta", "Margen"])

In [ ]:
#exploro la segunda tabla
df_dos=pd.read_json("https://sedeaplicaciones.minetur.gob.es/ServiciosRESTCarburantes/PreciosCarburantes/Listados/ComunidadesAutonomas/")
df_dos.head(10)


,IDCCAA,CCAA
0,1,Andalucia
1,2,Aragón
2,3,Asturias
3,4,Baleares
4,5,Canarias
5,6,Cantabria
6,7,Castilla la Mancha
7,8,Castilla y León
8,9,Cataluña
9,10,Comunidad Valenciana


In [ ]:
df_dos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   IDCCAA  19 non-null     int64 
 1   CCAA    19 non-null     object
dtypes: int64(1), object(1)
memory usage: 432.0+ bytes


In [ ]:
#cambiamos el type de IDCCAA a string
df_dos=df_dos.assign(IDCCAA= lambda dataset: dataset.IDCCAA.astype("str"))
df_dos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   IDCCAA  19 non-null     object
 1   CCAA    19 non-null     object
dtypes: object(2)
memory usage: 432.0+ bytes


In [ ]:
#utilizamos la función zfill para añadir ceros delante del ID y dejar los valores IDCAA de df_dos
#igual que la columna IDCAA de df_uno, ya que es por donde vamos a unir ambas tablas
df_dos['IDCCAA'] = df_dos['IDCCAA'].str.zfill(2)
df_dos

,IDCCAA,CCAA
0,01,Andalucia
1,02,Aragón
2,03,Asturias
3,04,Baleares
4,05,Canarias
5,06,Cantabria
6,07,Castilla la Mancha
7,08,Castilla y León
8,09,Cataluña
9,10,Comunidad Valenciana


**Unión de las dos tablas, tabla final**

In [ ]:
#Uno las dos tablas
df_final=df_uno.join(df_dos.set_index('IDCCAA'), on='IDCCAA')
df_final

,C.P.,Latitud,Localidad,Longitud (WGS84),Municipio,Precio Biodiesel,Precio Bioetanol,Precio Gas Natural Comprimido,Precio Gas Natural Licuado,Precio Gases licuados del petróleo,Precio Gasoleo A,Precio Gasoleo B,Precio Gasoleo Premium,Precio Gasolina 95 E10,Precio Gasolina 95 E5,Precio Gasolina 95 E5 Premium,Precio Gasolina 98 E10,Precio Gasolina 98 E5,Precio Hidrogeno,Provincia,IDEESS,IDMunicipio,IDProvincia,IDCCAA,CCAA
0,02250,"39,211417",ABENGIBRE,"-1,539167",Abengibre,,,,,,"1,349","0,879",,,"1,459",,,,,ALBACETE,4375,52,02,07,Castilla la Mancha
1,02152,"39,100389",ALATOZ,"-1,346083",Alatoz,,,,,,"1,390",,"1,500",,"1,519",,,"1,600",,ALBACETE,5122,53,02,07,Castilla la Mancha
2,02001,"39,054694",ALBACETE,"-1,832000",Albacete,,,,,,"1,419",,"1,509",,"1,509",,,"1,651",,ALBACETE,12054,54,02,07,Castilla la Mancha
3,02001,"39,000861",ALBACETE,"-1,849833",Albacete,,,,,,"1,279",,,,"1,389",,,,,ALBACETE,13933,54,02,07,Castilla la Mancha
4,02001,"38,985667",ALBACETE,"-1,868500",Albacete,,,,,,"1,399",,"1,429",,"1,499",,,"1,639",,ALBACETE,10765,54,02,07,Castilla la Mancha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11403,50197,"41,637944",ZARAGOZA,"-0,986389",Zaragoza,,,,,,"1,409","1,036",,,,,,,,ZARAGOZA,12472,8108,50,02,Aragón
11404,50197,"41,645722",ZARAGOZA,"-0,974194",Zaragoza,,,,,,"1,359",,,,,,,,,ZARAGOZA,13917,8108,50,02,Aragón
11405,50800,"41,857000",ZUERA,"-0,794000",Zuera,,,,,,"1,399","1,001","1,489",,"1,519",,,"1,649",,ZARAGOZA,1452,8109,50,02,Aragón
11406,50800,"41,883583",ZUERA,"-0,782833",Zuera,,,,,,"1,306","0,775","1,346",,"1,459",,,,,ZARAGOZA,9085,8109,50,02,Aragón


In [ ]:
#Exploro la tabla para ver lo tipos y las columnas que tengo que estrechar
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11408 entries, 0 to 11407
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   C.P.                                11408 non-null  object
 1   Latitud                             11408 non-null  object
 2   Localidad                           11408 non-null  object
 3   Longitud (WGS84)                    11408 non-null  object
 4   Municipio                           11408 non-null  object
 5   Precio Biodiesel                    11408 non-null  object
 6   Precio Bioetanol                    11408 non-null  object
 7   Precio Gas Natural Comprimido       11408 non-null  object
 8   Precio Gas Natural Licuado          11408 non-null  object
 9   Precio Gases licuados del petróleo  11408 non-null  object
 10  Precio Gasoleo A                    11408 non-null  object
 11  Precio Gasoleo B                    11408 non-null  ob

In [ ]:
#Estrechamos la tabla 

df_final=df_final.melt(id_vars= ["C.P.", "Latitud", "Localidad", "Longitud (WGS84)", "Municipio", "Provincia", "IDEESS", "IDMunicipio", "IDProvincia", "IDCCAA", "CCAA"], var_name="TipoPrecio", value_name= "Precio")
df_final

,C.P.,Latitud,Localidad,Longitud (WGS84),Municipio,Provincia,IDEESS,IDMunicipio,IDProvincia,IDCCAA,CCAA,TipoPrecio,Precio
0,02250,"39,211417",ABENGIBRE,"-1,539167",Abengibre,ALBACETE,4375,52,02,07,Castilla la Mancha,Precio Biodiesel,
1,02152,"39,100389",ALATOZ,"-1,346083",Alatoz,ALBACETE,5122,53,02,07,Castilla la Mancha,Precio Biodiesel,
2,02001,"39,054694",ALBACETE,"-1,832000",Albacete,ALBACETE,12054,54,02,07,Castilla la Mancha,Precio Biodiesel,
3,02001,"39,000861",ALBACETE,"-1,849833",Albacete,ALBACETE,13933,54,02,07,Castilla la Mancha,Precio Biodiesel,
4,02001,"38,985667",ALBACETE,"-1,868500",Albacete,ALBACETE,10765,54,02,07,Castilla la Mancha,Precio Biodiesel,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159707,50197,"41,637944",ZARAGOZA,"-0,986389",Zaragoza,ZARAGOZA,12472,8108,50,02,Aragón,Precio Hidrogeno,
159708,50197,"41,645722",ZARAGOZA,"-0,974194",Zaragoza,ZARAGOZA,13917,8108,50,02,Aragón,Precio Hidrogeno,
159709,50800,"41,857000",ZUERA,"-0,794000",Zuera,ZARAGOZA,1452,8109,50,02,Aragón,Precio Hidrogeno,
159710,50800,"41,883583",ZUERA,"-0,782833",Zuera,ZARAGOZA,9085,8109,50,02,Aragón,Precio Hidrogeno,


In [ ]:
#Ponemos bien los nombres de las columnas que sea necesario
df_final= df_final.rename(columns={"C.P.": "CodPos", "Longitud (WGS84)": "Longitud"})
df_final= df_final.filter(["IDEESS", "IDCCAA", "IDMunicipio", "Municipio", "IDProvincia", "Provincia", "Localidad", "CodPos",
         "Latitud", "Longitud", "CCAA", "TipoPrecio", "Precio"])
df_final


,IDEESS,IDCCAA,IDMunicipio,Municipio,IDProvincia,Provincia,Localidad,CodPos,Latitud,Longitud,CCAA,TipoPrecio,Precio
0,4375,07,52,Abengibre,02,ALBACETE,ABENGIBRE,02250,"39,211417","-1,539167",Castilla la Mancha,Precio Biodiesel,
1,5122,07,53,Alatoz,02,ALBACETE,ALATOZ,02152,"39,100389","-1,346083",Castilla la Mancha,Precio Biodiesel,
2,12054,07,54,Albacete,02,ALBACETE,ALBACETE,02001,"39,054694","-1,832000",Castilla la Mancha,Precio Biodiesel,
3,13933,07,54,Albacete,02,ALBACETE,ALBACETE,02001,"39,000861","-1,849833",Castilla la Mancha,Precio Biodiesel,
4,10765,07,54,Albacete,02,ALBACETE,ALBACETE,02001,"38,985667","-1,868500",Castilla la Mancha,Precio Biodiesel,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159707,12472,02,8108,Zaragoza,50,ZARAGOZA,ZARAGOZA,50197,"41,637944","-0,986389",Aragón,Precio Hidrogeno,
159708,13917,02,8108,Zaragoza,50,ZARAGOZA,ZARAGOZA,50197,"41,645722","-0,974194",Aragón,Precio Hidrogeno,
159709,1452,02,8109,Zuera,50,ZARAGOZA,ZUERA,50800,"41,857000","-0,794000",Aragón,Precio Hidrogeno,
159710,9085,02,8109,Zuera,50,ZARAGOZA,ZUERA,50800,"41,883583","-0,782833",Aragón,Precio Hidrogeno,


In [ ]:
#Limpiar comas y guiones de Latitud, Precio y Longitud. Donde lleva una coma, deberían llevar un punto.
#Los convierto en float
#Quito las filas con valor nulo en la columna Precio

df_final=df_final.replace(regex={r',': r'.'})

df_final=df_final\
.assign(Latitud= lambda dataset: dataset.Latitud.astype("float"))\
.assign(Longitud= lambda dataset: dataset.Longitud.astype("float"))\
.assign(Precio= lambda dataset: pd.to_numeric(dataset.Precio, errors= 'coerce'))\
.dropna(subset=['Precio'])

df_final


,IDEESS,IDCCAA,IDMunicipio,Municipio,IDProvincia,Provincia,Localidad,CodPos,Latitud,Longitud,CCAA,TipoPrecio,Precio
227,3731,10,152,Alicante/Alacant,03,ALICANTE,ALICANTE/ALACANT,03008,38.335694,-0.512833,Comunidad Valenciana,Precio Biodiesel,1.348
821,16,16,7,Arrazua-Ubarrundia,01,ARABA/ÁLAVA,ZURBANO,01520,42.876861,-2.616361,País Vasco,Precio Biodiesel,1.529
841,11651,16,39,Ribera Baja/Erribera Beitia,01,ARABA/ÁLAVA,RIBABELLOSA,01213,42.692917,-2.920667,País Vasco,Precio Biodiesel,1.229
854,12416,16,46,Vitoria-Gasteiz,01,ARABA/ÁLAVA,VITORIA-GASTEIZ,01010,42.863778,-2.712722,País Vasco,Precio Biodiesel,1.529
1144,4919,08,396,Barco de Ávila (El),05,ÁVILA,BARCO DE AVILA (EL),05600,40.358500,-5.516528,Castilla y León,Precio Biodiesel,1.429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148293,1483,02,8108,Zaragoza,50,ZARAGOZA,GARRAPINILLOS,50190,41.663639,-0.983306,Aragón,Precio Gasolina 98 E5,1.639
148294,11519,02,8108,Zaragoza,50,ZARAGOZA,GARRAPINILLOS,50190,41.651444,-0.981111,Aragón,Precio Gasolina 98 E5,1.519
148297,8275,02,8108,Zaragoza,50,ZARAGOZA,VILLARRAPA,50692,41.740389,-1.078944,Aragón,Precio Gasolina 98 E5,1.659
148298,12289,02,8108,Zaragoza,50,ZARAGOZA,ZARAGOZA,50197,41.636528,-0.990111,Aragón,Precio Gasolina 98 E5,1.679


## Verificación de datos

In [ ]:
#Comprobamos el tipo de las columnas
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39611 entries, 227 to 148301
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   IDEESS       39611 non-null  object 
 1   IDCCAA       39611 non-null  object 
 2   IDMunicipio  39611 non-null  object 
 3   Municipio    39611 non-null  object 
 4   IDProvincia  39611 non-null  object 
 5   Provincia    39611 non-null  object 
 6   Localidad    39611 non-null  object 
 7   CodPos       39611 non-null  object 
 8   Latitud      39611 non-null  float64
 9   Longitud     39611 non-null  float64
 10  CCAA         39611 non-null  object 
 11  TipoPrecio   39611 non-null  object 
 12  Precio       39611 non-null  float64
dtypes: float64(3), object(10)
memory usage: 4.2+ MB


In [ ]:
df_final.query("IDEESS == '14187'")

,IDEESS,IDCCAA,IDMunicipio,Municipio,IDProvincia,Provincia,Localidad,CodPos,Latitud,Longitud,CCAA,TipoPrecio,Precio
62058,14187,01,2786,Motril,18,GRANADA,MOTRIL,18600,36.770722,-3.55725,Andalucia,Precio Gasoleo A,1.399
84874,14187,01,2786,Motril,18,GRANADA,MOTRIL,18600,36.770722,-3.55725,Andalucia,Precio Gasoleo Premium,1.465
107690,14187,01,2786,Motril,18,GRANADA,MOTRIL,18600,36.770722,-3.55725,Andalucia,Precio Gasolina 95 E5,1.534
119098,14187,01,2786,Motril,18,GRANADA,MOTRIL,18600,36.770722,-3.55725,Andalucia,Precio Gasolina 95 E5 Premium,1.594
141914,14187,01,2786,Motril,18,GRANADA,MOTRIL,18600,36.770722,-3.55725,Andalucia,Precio Gasolina 98 E5,1.666


In [ ]:
df_final.TipoPrecio.value_counts()

Precio Gasoleo A                      11083
Precio Gasolina 95 E5                 10392
Precio Gasoleo Premium                 7439
Precio Gasolina 98 E5                  6005
Precio Gasoleo B                       2975
Precio Gases licuados del petróleo      782
Precio Gasolina 95 E5 Premium           701
Precio Gas Natural Comprimido           106
Precio Gas Natural Licuado               69
Precio Biodiesel                         36
Precio Gasolina 95 E10                   14
Precio Gasolina 98 E10                    5
Precio Bioetanol                          4
Name: TipoPrecio, dtype: int64